<a href="https://colab.research.google.com/github/1013115/KTB_Project/blob/main/%EC%9D%B4%EB%AF%B8%EC%A7%80%EC%9D%B8%EC%A6%9D_%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain

In [ ]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.0 MB/s eta 0:00:00


In [ ]:
!pip install pyheif pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 35.1 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

In [ ]:
image_path = "https://kinto.kr/data/item/20942/thumb-1602706457_9227_20942_1024x1024_600x600.jpg" #텀블러사진
#image_path="https://pds.joongang.co.kr//news/component/htmlphoto_mmdata/201807/22/b1c26d54-895f-487f-a687-33f94e3b06ad.jpg" # 양산 쓴 사진
#image_path = "https://www.dhresource.com/webp/m/0x0/f2/albu/g15/M00/3C/E5/rBVa3V96yGyAXYf9AAECpkmzDLU924.jpg" # 접혀있는 양산 사진
#image_path = "https://image.dongascience.com/Photo/2017/07/14993952480161.jpg" # 산책로 사진

mission = "날씨가 더우니 텀블러를 들고 시원한 음료를 마셔봐요."

In [ ]:
import os
import base64
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
# Vision 모델을 사용하기 위한 ChatOpenAI 클래스의 인스턴스를 생성합니다.
vision_model = ChatOpenAI(
    model="gpt-4o-mini", temperature=0
)

prompt = [
    SystemMessage(content = "I am an good image analyst."),
    HumanMessage(content=[
        {"type":"text", "text":"Describe the given image in detail"},
        {
            "type":"image_url",
            "image_url":{
                "url": f"{image_path}"
            }
        }
    ]),

]
prompt = ChatPromptTemplate.from_messages(prompt)
chain1 = LLMChain(prompt=prompt, llm=vision_model, output_key="content", verbose=True)


In [ ]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

class image(BaseModel):
    content: list[str] = Field(description="설명")

output_parser = PydanticOutputParser(pydantic_object=image)

template2 = """
You are an translator.
Please translate the following sentence into Korean.

{format_instructions}

content : {content}
"""

prompt2 = PromptTemplate(
    input_variables=["content"],
    template=template2,
    partial_variables={"format_instructions":output_parser.get_format_instructions()}
)

chain2 = LLMChain(prompt=prompt2, llm=vision_model,output_key="translated",verbose=True)

In [ ]:
template3 = """
Given the following mission: {mission} and the image translated: {translated} , determine if the image description fulfills the mission. Respond with `true` if it does, otherwise `false`.

"""

prompt3 = PromptTemplate(
    input_variables=["translated","mission"],
    template=template3,
)

chain3 =LLMChain(prompt=prompt3, llm=vision_model, output_key="result",verbose=True)


In [ ]:
from langchain.chains import SequentialChain

final_chain = SequentialChain(chains = [chain1, chain2, chain3], input_variables = ["image_path", "mission"],verbose=True)

In [ ]:
final_chain.run({"image_path":image_path, "mission":mission})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: I am an good image analyst.
Human: [{'type': 'text', 'text': 'Describe the given image in detail'}, {'type': 'image_url', 'image_url': {'url': 'https://kinto.kr/data/item/20942/thumb-1602706457_9227_20942_1024x1024_600x600.jpg'}}]

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an translator.
Please translate the following sentence into Korean.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"content": {"description": "설명", "i

'true'